In [3]:
#!/usr/bin/env python
# -*- coding:UTF-8 -*-

# from comet_ml import Experiment
import glob
import argparse
import os
import time
import tensorflow as tf
from model import RPN3D
from config import cfg
from utils import *
from utils.kitti_loader import iterate_data, sample_test_data,GPULogging
import subprocess


/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
ctx = 'urban' 
tag = 'default_pedestrian'
clss=cfg.DETECT_OBJ
output_path = tag+clss+ctx

exp = 'urban'
bs=1
vis=False
GPU_AVAILABLE='0'
GPU_USE_COUNT=1

In [ ]:
gpu_log= GPULogging(name=f'vox{ctx}-{clss}',device_id=GPU_AVAILABLE)


In [ ]:
!nvidia-smi

In [ ]:

path=''
dataset_dir = cfg.DATA_DIR
val_dir = os.path.join(cfg.CONTEXT_DIR, ctx)
save_model_dir = os.path.join('./save_model', tag)
total_inference_time = []
# experiment = Experiment(api_key="xXtJguCo8yFdU7dpjEpo6YbHw",project_name=exp)
# create output folder
os.makedirs(output_path, exist_ok=True)
os.makedirs(os.path.join(output_path, 'data'), exist_ok=True)
if vis:
    os.makedirs(os.path.join(output_path, 'vis'), exist_ok=True)
gpu_log.start()
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=cfg.GPU_MEMORY_FRACTION,
                        visible_device_list=GPU_AVAILABLE,
                        allow_growth=True)

    config = tf.ConfigProto(
        gpu_options=gpu_options,
        device_count={
            "GPU": GPU_USE_COUNT,
        },
        allow_soft_placement=True,
        log_device_placement=False
    )

    with tf.Session(config=config) as sess:
        model = RPN3D(
            cls=cfg.DETECT_OBJ,
            single_batch_size=bs,
            avail_gpus=GPU_AVAILABLE
        )
        if tf.train.get_checkpoint_state(save_model_dir):
            print("Reading model parameters from %s" % save_model_dir)
            model.saver.restore(
                sess, tf.train.latest_checkpoint(save_model_dir))
        counter=0
#         with experiment.test():
        for batch in iterate_data(val_dir, shuffle=False, aug=False, is_testset=False, batch_size=bs * GPU_USE_COUNT, multi_gpu_sum=GPU_USE_COUNT):
#             experiment.log_metric("counter",counter)

            if vis:
                tags, results, front_images, bird_views, heatmaps = model.predict_step(sess, batch, summary=False, vis=True)
            else:
                inference_start_time=time.time()
                tags, results = model.predict_step(sess, batch, summary=False, vis=False)
                inference_time = time.time()-inference_start_time
                total_inference_time.append(inference_time)
            # ret: A, B
            # A: (N) tag
            # B: (N, N') (class, x, y, z, h, w, l, rz, score)
            for tag, result in zip(tags, results):
                of_path = os.path.join(output_path, 'data', tag + '.txt')
                with open(of_path, 'w+') as f:
                    labels = box3d_to_label([result[:, 1:8]], [result[:, 0]], [result[:, -1]], coordinate='lidar')[0]
                    for line in labels:
                        f.write(line)
#                     print('write out {} objects to {}'.format(len(labels), tag))
            # dump visualizations
            if vis:
                for tag, front_image, bird_view, heatmap in zip(tags, front_images, bird_views, heatmaps):
                    front_img_path = os.path.join( output_path, 'vis', tag + '_front.jpg'  )
                    bird_view_path = os.path.join( output_path, 'vis', tag + '_bv.jpg'  )
                    heatmap_path = os.path.join( output_path, 'vis', tag + '_heatmap.jpg'  )
                    cv2.imwrite( front_img_path, front_image )
                    cv2.imwrite( bird_view_path, bird_view )
                    cv2.imwrite( heatmap_path, heatmap )
            counter +=1
            
gpu_log.stop()
# process.kill()
# log.flush()
# log.close()

In [ ]:
print(counter)

In [ ]:
mean_inference = np.mean(total_inference_time)
min_inference = np.min(total_inference_time)
max_inference =np.max(total_inference_time)
var_inference = np.var(total_inference_time)

print('Min: %fs Max: %fs Mean: %fs Var: %fs ' %( min_inference,max_inference,mean_inference,var_inference))

In [5]:
# execute evaluation code
cmd_1 = "./kitti_eval/evaluate_object_3d_offline"
cmd_2 = f"/mnt/storage/home/ja17618/scratch/DATA_DIR/context/{ctx}/label_2/"
cmd_3 = "/mnt/storage/home/ja17618/scratch/voxelnet/"+output_path
# cmd_3 = os.path.join( output_path, 'log' )
var= os.popen(" ".join( [cmd_1, cmd_2,cmd_3] )).read()


AttributeError: 'str' object has no attribute 'decode'

In [11]:
var.encode('utf-8').decode('unicode_escape')

"Thank you for participating in our evaluation!\nLoading detections...\nnumber of files for evaluation: 1029\n  done.\nstart to eval image 2D bounding boxes... \nsave /mnt/storage/home/ja17618/scratch/voxelnet/default_pedestrianPedestrianurban/plot/pedestrian_detection.txt\npedestrian_detection AP: 72.129387 65.245384 65.508232\n2D bounding boxes detection saved ... \n  start to compute aos ... \nsave /mnt/storage/home/ja17618/scratch/voxelnet/default_pedestrianPedestrianurban/plot/pedestrian_orientation.txt\npedestrian_orientation AP: 35.719177 32.809044 32.804714\n2D bounding boxes aos saved ... \nstart to eval bird's eye view bounding boxes... \nsave /mnt/storage/home/ja17618/scratch/voxelnet/default_pedestrianPedestrianurban/plot/pedestrian_detection_ground.txt\npedestrian_detection_ground AP: 74.766678 73.944069 74.138588\nstart to eval 3D bounding boxes... \nsave /mnt/storage/home/ja17618/scratch/voxelnet/default_pedestrianPedestrianurban/plot/pedestrian_detection_3d.txt\npedestr